In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Dropout 
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist 

In [3]:
(x_train, y_train), (x_test, y_test)=mnist.load_data() 
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

print(f"Training data shape: {x_train.shape}")
print(f"Testing data shape: {x_test.shape}")

Training data shape: (60000, 28, 28)
Testing data shape: (10000, 28, 28)


In [6]:
def build_model(learning_rate=0.001, hidden_units=128):
    model=Sequential([
        Flatten(input_shape=(28,28)),
        Dense(hidden_units,activation='relu'),
        Dropout(0.2),
        Dense(10,activation='softmax')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [7]:
configs = [
    {'lr': 0.001, 'batch': 32, 'units': 64},
    {'lr': 0.001, 'batch': 64, 'units': 128},
    {'lr': 0.0005, 'batch': 32, 'units': 128},
]


In [9]:
import os 
from datetime import datetime
results = []

for cfg in configs:
    run_name = f"lr{cfg['lr']}_bs{cfg['batch']}_units{cfg['units']}_{datetime.now().strftime('%H%M%S')}"
    log_dir = os.path.join("logs", run_name)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    print(f"Training model with {cfg} ...")

    model = build_model(cfg['lr'], cfg['units'])
    history = model.fit(
        x_train, y_train,
        epochs=5,
        batch_size=cfg['batch'],
        validation_data=(x_test, y_test),
        callbacks=[tensorboard_callback],
        verbose=0
    )

    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    results.append({'config': cfg, 'accuracy': test_acc, 'loss': test_loss})

    print(f"Done! Accuracy={test_acc:.4f}, Loss={test_loss:.4f}\n")


Training model with {'lr': 0.001, 'batch': 32, 'units': 64} ...


c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Done! Accuracy=0.9726, Loss=0.0933

Training model with {'lr': 0.001, 'batch': 64, 'units': 128} ...


c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Done! Accuracy=0.9750, Loss=0.0780

Training model with {'lr': 0.0005, 'batch': 32, 'units': 128} ...
✅ Done! Accuracy=0.9772, Loss=0.0812



In [10]:
print("Model Comparison Report:")
for res in results:
    cfg = res['config']
    print(f"LR={cfg['lr']}, Batch={cfg['batch']}, Units={cfg['units']} --> Accuracy={res['accuracy']:.4f}")


Model Comparison Report:
LR=0.001, Batch=32, Units=64 --> Accuracy=0.9726
LR=0.001, Batch=64, Units=128 --> Accuracy=0.9750
LR=0.0005, Batch=32, Units=128 --> Accuracy=0.9772
